<a href="https://colab.research.google.com/github/Otsemeuno/Customer-Subscription/blob/main/Customer_Subscription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Customer Subscription
$(Adapted from Kaggle)$


## Project Statement
This data is about a subscription-based digital product offering for financial advisory that includes newsletters, webinars, and investment recommendations. The offering has a couple of varieties, annual subscription, and digital subscription. The product also provides daytime support for customers to reach out to a care team that can help them with any product-related questions and signup/cancellation-related queries.

The data set contains the following information:

<ul><li>customer sign-up and cancellation dates at the product level
<li>call center activity
<li>customer demographics
<li>product pricing info<ul>

## Objectives

Forecast 2022 year revenue if the company expects to add the same number of new customers as the previous year

In 2022, one of the company's key initiatives is to improve the customer experience, one of the capabilities is superior customer care service, and they are thinking about enhancing in-product support better but that takes beyond 2022 to happen. It is very important to forecast call center case volume accurately so that they can make appropriate staffing decisions. Need to predict daily call volumes and make recommendations on how to staff on an hourly basis.

In [ ]:
#importing drive
from google.colab import drive

In [ ]:
#importing all necessary libraries
import pandas as pd
import numpy as np

##Data Preparation

####Load the dataset

In [ ]:
# Mount Google Drive on Kernel
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
customer_cases = pd.read_csv('/content/drive/MyDrive/Kaggle project/customer_cases.csv', index_col =[0])

In [25]:
customer_info = pd.read_csv('/content/drive/MyDrive/Kaggle project/customer_info.csv', index_col =[0])

In [26]:
customer_product = pd.read_csv('/content/drive/MyDrive/Kaggle project/customer_product.csv', index_col =[0])

In [27]:
product_info = pd.read_csv('/content/drive/MyDrive/Kaggle project/product_info.csv', index_col =[0])

In [28]:
customer_info.head()

,customer_id,age,gender
1,C2448,76,female
2,C2449,61,male
3,C2450,58,female
4,C2451,62,female
5,C2452,71,male
